# Study of the powerlaw exponent as a function of 8 micron intensity


In [1]:
import numpy as np
import astropy.io.fits as pyfits
import pandas as pd

import matplotlib.pyplot as plt

from scipy.stats import pearsonr

import support_functions as sup

In [2]:
#### User input ####

## information to fetch the necessary data
dir_path = "../data_CIIcorr/data_FEEDBACK/" ## path to the data directory
name_CII = "_CII_final_res20_grid8_0p5_clean_integrated" ## identifier of the [CII] data file
name_8m = "_Spitzer_IRAC_8_20_8" ## identifier of the 8 micron data file
plot_path = "../plots_CIIcorr/CII_correlations/"

list_regions = [ ## list of the regions to be studied
    "CYGNUSX",
    "M16",
    "M17",
    "NGC7538",
    "RCW36",
    "RCW49",
    "RCW79",
    "RCW120",
    "W40",
    #"ORIONA"
]

## minimal intensity for the [CII] data
min_cut_CII = 20.

## number of datapoints to be considered to fit a powerlaw exponent
num_data_points = 1000. #[500, 1000]

## minimum number of data points needed in the final interval to perform a fit
min_points_final = 50

####################

In [3]:
## Study the powerlaws for each region
for region in list_regions:
    print("Analyzing {reg}".format(reg = region))
    
    ## read the data into a Pandas DataFrame
    df = sup.read_data_to_DF(
        "{path}{region}/{region}{CII}.fits".format(path = dir_path, region = region, CII = name_CII),
        "{path}{region}/{region}{name}.fits".format(path = dir_path, region = region, name = name_8m),
        min_cut_CII,
        region
    )
    
    ## sort the DataFrame
    df = df.sort_values(by = ["8 micron"])
    
    ## loop over the different subsamples that will be fitted with a powerlaw
    num_groups = int(len(df.index)/num_data_points) + 1
    for i in range(0, num_groups):
        ## define the begin and end
        beg = int(i*num_data_points)
        end = int((i+1)*num_data_points)
        
        ## extract the sub sample
        sub_df = df.iloc[beg:end]
        
        ## fit the sub_sample
        popt, pcov = curve_fit(sub_df['8 micron'], sub_df['CII'])

Analyzing CYGNUSX
41
            CII  8 micron
4943  -3.701836 -2.021971
5046  -3.630631 -2.020520
15861 -3.556336 -2.019880
15862 -3.483972 -2.018976
4841  -3.711516 -2.018194
...         ...       ...
39394 -3.842441 -1.978998
7     -3.735030 -1.978992
483   -3.445704 -1.978961
9842  -3.481130 -1.978913
10519 -3.777557 -1.978894

[1000 rows x 2 columns]
            CII  8 micron
6367  -3.811078 -1.978894
209   -3.801092 -1.978843
9422  -3.589187 -1.978814
13757 -3.771387 -1.978792
2226  -3.446643 -1.978783
...         ...       ...
21    -3.558454 -1.967632
11075 -3.846444 -1.967617
7439  -3.737738 -1.967614
6370  -3.842098 -1.967609
9250  -3.391446 -1.967585

[1000 rows x 2 columns]
            CII  8 micron
12237 -3.763391 -1.967581
88    -3.536227 -1.967576
40311 -3.738606 -1.967559
305   -3.457291 -1.967557
85    -3.522643 -1.967548
...         ...       ...
40033 -3.732472 -1.959462
692   -3.741741 -1.959427
294   -3.503905 -1.959425
8075  -3.844486 -1.959420
9010  -3.562837 -1.